In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Load your Salesforce data (replace with your file path or DataFrame) ---
# Example CSV: 'salesforce_opportunities.csv'
df = pd.read_csv('salesforce_opportunities.csv', parse_dates=['Opportunity_Created'])

# --- Step 1: Get first interaction per customer ---
df['First_Interaction'] = df.groupby('Customer_ID')['Opportunity_Created'].transform('min')

# --- Step 2: Calculate months since first interaction ---
df['Months_Since_First_Interaction'] = ((df['Opportunity_Created'] - df['First_Interaction']) / pd.Timedelta(days=30)).round().astype(int)

# --- Step 3: Aggregate product inquiries by lifecycle stage ---
product_timeline = (
    df.groupby(['Product_Name', 'Months_Since_First_Interaction'])
    .size()
    .reset_index(name='Inquiry_Count')
)

# --- Step 4: Pivot for visualization ---
timeline_pivot = product_timeline.pivot(
    index='Months_Since_First_Interaction',
    columns='Product_Name',
    values='Inquiry_Count'
).fillna(0)

# --- Step 5: Plot heatmap ---
plt.figure(figsize=(14, 6))
sns.heatmap(timeline_pivot, cmap='YlGnBu', linewidths=0.5, annot=True, fmt='.0f')
plt.title('Customer Product Inquiries Over Lifecycle')
plt.xlabel('Product')
plt.ylabel('Months Since First Interaction')
plt.tight_layout()
plt.show()
